In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/0.1-initial-exploration.csv',index_col=False)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

**Drop useless columns**

In [3]:
df = df.drop(['resultId','raceId','constructorId','position','positionText','pointsGained',
'laps','time_x','milliseconds','fastestLapSpeed','fastestLapTime','fastestLap','statusId'
,'circuitName','date','dateOfDebut','status','driverStandingsId'], 1)

**Columns with null values**

In [4]:
print('Columns with null values:\n', df.columns[df.isna().any()].tolist())

Columns with null values:
 ['driverStandingsPoints', 'driverStandingsPosition', 'driverStandingsWins', 'lastRaceRank', 'constructorStandingsId', 'constructorStandingsPoints', 'constructorStandingsPosition', 'constructorStandingsWins', 'previousRaceGridStart', 'previousRacePosition', 'racesWon', 'racesRetired', 'racesFinished', 'polePositions', 'racesWonByConstructor', 'racesRetiredByConstructor', 'percentageOfBestQuali']


In [5]:
df[['racesWon','racesRetired','racesFinished','polePositions','racesWonByConstructor','racesRetiredByConstructor']] = df[['racesWon','racesRetired','racesFinished','polePositions','racesWonByConstructor','racesRetiredByConstructor']].fillna(value=0)
df[['lastRaceRank','previousRaceGridStart','previousRacePosition']] = df[['lastRaceRank','previousRaceGridStart','previousRacePosition']].fillna(value=-1)